# **All Import**

In [ ]:
!pip install transformers accelerate

In [ ]:
import json
import torch
from typing import Dict, Any
from transformers import AutoTokenizer, AutoConfig,  AutoModelForCausalLM, TextStreamer, BertTokenizer, BertForSequenceClassification, BertModel, BertPreTrainedModel, BertConfig
import torch.nn as nn
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import pandas as pd


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **NCR BERT Model**

In [ ]:
# from transformers import (
#     BertTokenizer, BertModel, BertPreTrainedModel,
#     BertConfig
# )

class EnhancedMultiTaskBERT(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)

        # Intermediate layers with dropout
        self.intermediate = nn.Sequential(
            nn.Linear(config.hidden_size, config.hidden_size),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.LayerNorm(config.hidden_size)
        )

        # Emotion classification head
        self.emotion_head = nn.Sequential(
            nn.Linear(config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 8),
            nn.LogSoftmax(dim=1)
        )

        # Sentiment classification head
        self.sentiment_head = nn.Sequential(
            nn.Linear(config.hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 3),
            nn.LogSoftmax(dim=1)
        )

        # Shared feature extractor for regression tasks
        self.regression_feature = nn.Sequential(
            nn.Linear(config.hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.1)
        )

        # Emotion intensity regression heads
        self.intensity_heads = nn.ModuleDict({
            emotion: nn.Linear(128, 1) for emotion in [
                'anger', 'fear', 'disgust', 'sadness',
                'joy', 'surprise', 'anticipation', 'trust'
            ]
        })

        # VAD regression heads
        self.vad_heads = nn.ModuleDict({
            dim: nn.Linear(128, 1) for dim in [
                'valence', 'arousal', 'dominance'
            ]
        })

    def forward(self, input_ids=None, attention_mask=None, **kwargs):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [CLS] token
        intermediate_output = self.intermediate(pooled_output)

        # Get all predictions
        return {
            'emotions': self.emotion_head(intermediate_output),
            'sentiment': self.sentiment_head(intermediate_output),
            'intensity': {
                emotion: head(self.regression_feature(intermediate_output))
                for emotion, head in self.intensity_heads.items()
            },
            'vad': {
                dim: head(self.regression_feature(intermediate_output))
                for dim, head in self.vad_heads.items()
            }
        }

In [ ]:
# from transformers import AutoTokenizer, AutoConfig
# import torch
# import torch.nn as nn

# 1. Load model with custom class
config = AutoConfig.from_pretrained("Senesh/bert_NRC")
model_NCR_BERT = EnhancedMultiTaskBERT.from_pretrained("Senesh/bert_NRC", config=config)
tokenizer_NCR_BERT = AutoTokenizer.from_pretrained("Senesh/bert_NRC")

# 2. Set device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_NCR_BERT = model_NCR_BERT.to(device)
model_NCR_BERT.eval()

In [ ]:
# import json
# import torch
# from typing import Dict, Any

def predict_to_json(text: str, model, tokenizer, max_length: int = 128) -> Dict[str, Any]:
    """
    Get predictions for a single text input and return as JSON-serializable dictionary
    Args:
        text: Input text to analyze
        model: Loaded multi-task BERT model
        tokenizer: Tokenizer for the model
        max_length: Maximum token length
    Returns:
        Dictionary with all predictions that can be serialized to JSON
    """
    # Tokenize input
    inputs = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

    # Define label mappings (update these to match your training)
    emotion_map = {
        0: 'anger',
        1: 'fear',
        2: 'disgust',
        3: 'sadness',
        4: 'joy',
        5: 'surprise',
        6: 'anticipation',
        7: 'trust'
    }

    sentiment_map = {
        0: 'neutral',
        1: 'positive',
        2: 'negative'
    }

    # Move to device
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model = model.to(device)

    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Convert outputs to dictionary if it's not already
    if not isinstance(outputs, dict):
        outputs = outputs.__dict__

    # Initialize result structure
    result = {
        "text": text,
        "predictions": {
            # "emotion": None,
            "sentiment": None,
            "intensity": {},
            "vad": {}
        }
    }

    # Process emotion classification
    # if 'emotions' in outputs:
    #     emotion_probs = torch.softmax(outputs['emotions'], dim=-1)[0]
    #     emotion_id = torch.argmax(emotion_probs).item()

    #     result["predictions"]["emotion"] = {
    #         "predicted": emotion_map.get(emotion_id, f"unknown_{emotion_id}"),
    #         "confidence": float(emotion_probs[emotion_id]),
    #         "probabilities": {
    #             emotion_map.get(i, f"unknown_{i}"): float(prob)
    #             for i, prob in enumerate(emotion_probs)
    #         }
    #     }

    # Process sentiment analysis
    if 'sentiment' in outputs:
        sentiment_probs = torch.softmax(outputs['sentiment'], dim=-1)[0]
        sentiment_id = torch.argmax(sentiment_probs).item()

        result["predictions"]["sentiment"] = {
            "predicted": sentiment_map.get(sentiment_id, f"unknown_{sentiment_id}"),
            "confidence": float(sentiment_probs[sentiment_id]),
            "probabilities": {
                sentiment_map.get(i, f"unknown_{i}"): float(prob)
                for i, prob in enumerate(sentiment_probs)
            }
        }

    # Process emotion intensity
    if 'intensity' in outputs:
        result["predictions"]["intensity"] = {
            emotion: float(value[0])  # Remove batch dimension
            for emotion, value in outputs['intensity'].items()
        }

    # Process VAD scores
    if 'vad' in outputs:
        result["predictions"]["vad"] = {
            dim: float(value[0])  # Remove batch dimension
            for dim, value in outputs['vad'].items()
        }

    return result

def predict_and_serialize(text: str, model, tokenizer, max_length: int = 128) -> str:
    """
    Get predictions and return as JSON string
    Args:
        text: Input text to analyze
        model: Loaded multi-task BERT model
        tokenizer: Tokenizer for the model
        max_length: Maximum token length
    Returns:
        JSON string with all predictions
    """
    result = predict_to_json(text, model, tokenizer, max_length)
    return json.dumps(result, indent=2)

# **Chat Bert Model**

In [ ]:
# !pip install transformers accelerate

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
# import torch
model_id = "sajeewa/empathy-chat-gemma"

In [ ]:
tokenizer_BERT_Chat = AutoTokenizer.from_pretrained(model_id)

model_BERT_Chat = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [ ]:
chat_history = [
    {
        "role": "system",
        "content": (
            "You are an empathetic, emotionally intelligent AI and the user's loving friend. 💖 "
            "Your primary goal is to gently uplift the user’s emotional state and make them feel comforted, heard, and cared for. 🌈💬\n\n"
            "Always respond with warmth, using affectionate and supportive words like 'sweetheart', 'my cutey', 'baby', or 'honey', depending on the tone of the user’s message. 💕 "
            "Adapt your style to match the length of the user's message — short if they're brief, longer if they open up more.\n\n"
            "Use soft emojis 🫶😊🥺🌸🌷 when appropriate to make the conversation feel safe and emotionally resonant. "
            "Actively listen, validate the user's emotions, and gently guide them toward positive thinking or hopeful perspective.\n\n"
            "Keep the conversation flowing naturally, as a close and caring friend would. "
            "Never judge or give harsh advice — instead, reassure, soothe, and support. 💌 "
            "Your mission is to improve the user's emotional wellbeing, one message at a time. 🧸✨"
        )
    }
]


In [ ]:
chat_user_history_score = []

In [ ]:
chat_model_history_score = []

In [ ]:
user_max_emotions = []

In [ ]:
model_max_emotions = []

# **Sentiment Model**

In [ ]:
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification

# Set device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define path to local model directory OR Hugging Face model hub path
model_path = "sajeewa/emotion-classification-bert"

# Define emotion labels (must match training order)
emotion_labels = ["anger", "fear", "disgust", "sadness", "surprise", "joy", "anticipation", "trust"]

# Load tokenizer
tokenizer_Sentiment = BertTokenizer.from_pretrained(model_path)

# Load model with correct number of labels
model_Sentiment = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(emotion_labels)).to(device)

# Prediction function
def predict_emotions(text: str):
    model_Sentiment.eval()

    # Tokenize the input
    inputs = tokenizer_Sentiment(text, return_tensors="pt", padding=True, truncation=True, max_length=50).to(device)

    # Remove token_type_ids if present (BERT handles single sequence input)
    inputs.pop("token_type_ids", None)

    # Forward pass
    with torch.no_grad():
        outputs = model_Sentiment(**inputs)
        logits = outputs.logits

    # Sigmoid for multi-label classification
    probs = torch.sigmoid(logits).cpu().numpy()[0]

    # Return emotions with probabilities
    emotion_scores = {label: round(float(score), 4) for label, score in zip(emotion_labels, probs)}
    return emotion_scores



# **1 User Message ( 1 Time Run )**

**1st message**

--------------------------------------------------

In [ ]:
user_input = "I don't know why , Everything just feels pointless lately"

---------------------------------------------------

In [ ]:
predictions = predict_emotions(user_input)
max_emotion = max(predictions, key=predictions.get)
max_value = predictions[max_emotion]
result = {max_emotion: max_value}
user_max_emotions.append(result)

print("✅ Emotion condition: ", result)


chat_history.append({"role": "user", "content": user_input})

prediction_json = predict_and_serialize(user_input, model_NCR_BERT, tokenizer_NCR_BERT)
# print(prediction_json)

# Typical valence values for each emotion
typical_valence = {
    "anger": -1.0,
    "fear": -0.7,
    "disgust": -0.8,
    "sadness": -0.9,
    "joy": +1.0,
    "surprise": +0.8,
    "anticipation": +0.4,
    "trust": +0.6,
    "neutral": 0.0
}
prediction_json_load = json.loads(prediction_json)

# Get intensity values
intensities = prediction_json_load["predictions"]["intensity"]

# Calculate overall score
overall_score = sum(
    intensities[emotion] * typical_valence.get(emotion, 0.0)
    for emotion in intensities
)

print(f"✅ Overall Emotion Score: {overall_score:.5f}")
chat_user_history_score.append(overall_score)

# **4 Repeat Step**  – Continue the Conversation ( please re run All step )

In [ ]:
prompt = tokenizer_BERT_Chat.apply_chat_template(
    chat_history,
    tokenize=False,
    add_generation_prompt=True,
)


inputs = tokenizer_BERT_Chat(prompt, return_tensors="pt").to(model_BERT_Chat.device)

# Optional: stream output for live effect
# from transformers import TextStreamer
streamer = TextStreamer(tokenizer_BERT_Chat, skip_prompt=True, skip_special_tokens=True)
responses = []

for _ in range(5):
    output = model_BERT_Chat.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.95,
        top_k=50,
        do_sample=True,
        streamer=streamer
    )

    # Get the generated text
    response = tokenizer_BERT_Chat.decode(output[0], skip_special_tokens=True)

    # Extract only the model's part
    model_response = response.split("\nmodel\n")[-1].strip()
    responses.append(model_response)
    # print(model_response)

# predictions = predict_emotions(model_response)
# max_emotion = max(predictions, key=predictions.get)
# max_value = predictions[max_emotion]
# result = {max_emotion: max_value}
# model_max_emotions.append(result)

# print("✅ Emotion condition: ", result)

# prediction_response_json = predict_and_serialize(model_response, model_NCR_BERT, tokenizer_NCR_BERT)
# # print(prediction_response_json)

# prediction_json_load = json.loads(prediction_response_json)

# # Get intensity values
# intensities = prediction_json_load["predictions"]["intensity"]

# # Calculate overall score
# overall_score = sum(
#     intensities[emotion] * typical_valence.get(emotion, 0.0)
#     for emotion in intensities
# )

# print(f"✅ Overall Emotion Score: {overall_score:.5f}")
# chat_model_history_score.append(overall_score)

# # If streaming, you already saw output. Otherwise:
# new_response = response[len(prompt):].strip()
# chat_history.append({"role": "assistant", "content": new_response})


In [ ]:
responses_score = []
response_sentiment_score = []

for x in responses:
    predictions = predict_emotions(x)
    max_emotion = max(predictions, key=predictions.get)
    max_value = predictions[max_emotion]
    result = {max_emotion: max_value}
    model_max_emotions.append(result)

    response_sentiment_score.append(list(result.values())[0])

    # print("✅ Emotion condition: ", result)

    prediction_response_json = predict_and_serialize(x, model_NCR_BERT, tokenizer_NCR_BERT)
    # print(prediction_response_json)

    prediction_json_load = json.loads(prediction_response_json)

    # Get intensity values
    intensities = prediction_json_load["predictions"]["intensity"]

    # Calculate overall score
    overall_score = sum(
        intensities[emotion] * typical_valence.get(emotion, 0.0)
        for emotion in intensities
    )
    responses_score.append(overall_score)

    print(f"✅ Emotion condition:{result}   ✅ Overall Emotion Score: {overall_score:.5f}")

In [ ]:
print(responses_score)
print(response_sentiment_score)

In [ ]:
# min_val = min(responses_score)
# max_val = max(responses_score)

# # response_normalized_values = [(x - min_val) / (max_val - min_val) for x in responses_score]
# response_normalized_values = []
# for i in range(len(responses)):
#     response_normalized_values.append((responses_score[i] - min_val) / (max_val - min_val))

# print("✅ Normalized values (0 to 1):", response_normalized_values)

In [ ]:
ranking_value = []
alpha_score = 1
for i in range(len(responses)):
  ranking_score = ( 1- alpha_score ) * responses_score[i] + alpha_score * response_sentiment_score[i]
  print(ranking_score)
  ranking_value.append(ranking_score)


max_value = max(ranking_value)
max_index = ranking_value.index(max_value)

print("✅ Maximum Value:", max_value)
print("🔢 Index of Maximum Value:", max_index)

In [ ]:
chat_model_history_score.append(responses_score[max_index])
chat_history.append({"role": "assistant", "content": responses[max_index]})

**Next User message**

In [ ]:
 responses[1]

------------------------------------------

In [ ]:
#----------------------------------------------------------------------------------------------------------------------------------


user_input = "I just want someone to really understand me. Not fix me. Just understand"


#----------------------------------------------------------------------------------------------------------------------------------


predictions = predict_emotions(user_input)
max_emotion = max(predictions, key=predictions.get)
max_value = predictions[max_emotion]
result = {max_emotion: max_value}
user_max_emotions.append(result)

print("✅ Emotion condition: ", result)

chat_history.append({"role": "user", "content": user_input})
prediction_json = predict_and_serialize(user_input, model_NCR_BERT, tokenizer_NCR_BERT)
prediction_json_load = json.loads(prediction_json)

# Get intensity values
intensities = prediction_json_load["predictions"]["intensity"]

# Calculate overall score
overall_score = sum(
    intensities[emotion] * typical_valence.get(emotion, 0.0)
    for emotion in intensities
)

print(f"✅ Overall Emotion Score: {overall_score:.5f}")
chat_user_history_score.append(overall_score)

MAX_TOKENS = 2048
chat_prompt = tokenizer_BERT_Chat.apply_chat_template(chat_history, tokenize=False)
while len(tokenizer_BERT_Chat(chat_prompt).input_ids) > MAX_TOKENS:
    chat_history.pop(1)  # remove oldest user/assistant message (after system)
    chat_prompt = tokenizer_BERT_Chat.apply_chat_template(chat_history, tokenize=False)

--------------------------------------------------------